# 차트만들기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set()

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)


In [ ]:
# 한글폰트 사용 in colab
%matplotlib inline  

import matplotlib as mpl 
import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm  

!apt-get update -qq
!apt-get install fonts-nanum* -qq

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' 
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()
mpl.rcParams['axes.unicode_minus'] = False

Selecting previously unselected package fonts-nanum.
(Reading database ... 155629 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...
NanumBarunGothic


In [ ]:
# ! pip install pandas-datareader # 주식 데이터 조회
# ! pip install plotly # 차트 라이브러리

In [ ]:
# ! pip install pykrx

In [ ]:
# 주식 데이터 조회 ( 삼성전자 )
import plotly.graph_objects as go
import plotly.subplots as ms
import pandas_datareader as web
from pykrx import stock
from pykrx import bond
import numpy as np
import pandas as pd
df = stock.get_market_ohlcv("20050101", "20200101", "004000")

df = df.astype(int) # Object 데이터를 int로 변환
df.columns = ['Open','High','Low','Close','Volume']
#이동평균선

df['MA5'] = df['Close'].rolling(5).mean()
df['MA20'] = df['Close'].rolling(20).mean()
df['MA60'] = df['Close'].rolling(60).mean()

# 캔들 차트 객체 생성
candle = go.Candlestick(
	x=df.index,
	open=df['Open'],
	high=df['High'],
	low=df['Low'],
	close=df['Close'],
    increasing_line_color = 'red', # 상승봉 스타일링
    decreasing_line_color = 'blue' # 하락봉 스타일링
)
#이동평균선 생성
ma5 = go.Scatter(x=df.index, y=df['MA5'], line=dict(color='black', width=0.8), name='ma5')
ma20 = go.Scatter(x=df.index, y=df['MA20'], line=dict(color='red', width=0.9), name='ma20')
ma60 = go.Scatter(x=df.index, y=df['MA60'], line=dict(color='green', width=1), name='ma60')


#RSI 구하기 
U = np.where(df['Close'].diff(1) > 0, df['Close'].diff(1), 0) 
D = np.where(df['Close'].diff(1) < 0, df['Close'].diff(1) *(-1), 0) 
AU = pd.DataFrame(U, index=df.index).rolling(window=14).mean() 
AD = pd.DataFrame(D, index=df.index).rolling(window=14).mean() 
RSI = AU / (AD+AU) *100 
df['RSI'] = RSI
RSI = go.Scatter(x=df.index, y=df['RSI'], line=dict(color='red', width=2), name='RSI')


# 바 차트(거래량) 객체 생성
volume_bar = go.Bar(x=df.index, y=df['Volume'])

fig = ms.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(candle, row=1, col=1)
fig.add_trace(volume_bar, row=2, col=1)
fig.add_trace(ma5, row=1, col=1)
fig.add_trace(ma20, row=1, col=1)
fig.add_trace(ma60, row=1, col=1)
fig.add_trace(RSI, row=3, col=1)
# #그래프 객체 바인딩
# fig = go.Figure(data=[candle, ma5, ma20, ma60])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(step="all")
        ])
    )
)


fig.update_layout(
    title='stock price',
    yaxis1_title='Stock Price',
    yaxis2_title='Volume',
    # xaxis2_title='periods',
    xaxis1_rangeslider_visible=False,
    xaxis2_rangeslider_visible=False,
    xaxis3_rangeslider_visible=False,
    width=1000, height=1000
)


config = dict({'scrollZoom': True})

# 출처: https://sjblog1.tistory.com/45 [sjblog]
fig.show()

https://sjblog1.tistory.com/45

In [ ]:
# ! pip install mplfinance

In [ ]:
from pykrx import stock 
import pandas as pd 
import mplfinance as mpf
import numpy as np
import plotly.graph_objects as go 
import plotly.subplots as ms 
import plotly.express as px

In [ ]:
# # # 전체 종목코드와 종목명 가져오기 
# # stock_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")}) 
# # stock_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x)) 

# # stock_name의 2021년 주가 데이터 가져오기 
# stock_name = "삼성전자" 
# stock_from = "20181220" 
# stock_to = "20211220" 

# ticker = '005930'
# df = stock.get_market_ohlcv_by_date(fromdate=stock_from, todate=stock_to, ticker=ticker) 

# # 칼럼명을 영문명으로 변경 
# df = df.rename(columns={'시가':'Open', '고가':'High', '저가':'Low', '종가':'Close', '거래량':'Volume'}) 

# df["Close"]=df["Close"].apply(pd.to_numeric,errors="coerce")


In [ ]:
def plotly차트(stock_name,stock_from,stock_to,ticker):

    # stock_name의 2021년 주가 데이터 가져오기 
    stock_name = stock_name
    stock_from = stock_from
    stock_to = stock_to

    ticker = ticker
    df = stock.get_market_ohlcv_by_date(fromdate=stock_from, todate=stock_to, ticker=ticker) 

    # 칼럼명을 영문명으로 변경 
    df = df.rename(columns={'시가':'Open', '고가':'High', '저가':'Low', '종가':'Close', '거래량':'Volume'}) 

    df["Close"]=df["Close"].apply(pd.to_numeric,errors="coerce")
    # 볼린저밴드 구하기 
    df['ma20'] = df['Close'].rolling(window=20).mean() # 20일 이동평균 
    df['stddev'] = df['Close'].rolling(window=20).std() # 20일 이동표준편차 
    df['upper'] = df['ma20'] + 2*df['stddev'] # 상단밴드 
    df['lower'] = df['ma20'] - 2*df['stddev'] # 하단밴드

    #이동평균선

    df['ma5'] = df['Close'].rolling(5).mean()
    # df['MA20'] = df['Close'].rolling(20).mean()
    df['ma60'] = df['Close'].rolling(60).mean()

    # MACD 구하기 
    df['ma12'] = df['Close'].rolling(window=12).mean() # 12일 이동평균 
    df['ma26'] = df['Close'].rolling(window=26).mean() # 26일 이동평균 
    df['MACD'] = df['ma12'] - df['ma26'] # MACD 
    df['MACD_Signal'] = df['MACD'].rolling(window=9).mean() # MACD Signal(MACD 9일 이동평균) 
    df['MACD_Oscil'] = df['MACD'] - df['MACD_Signal'] #MACD 오실레이터

    # 스토캐스틱 구하기 
    df['ndays_high'] = df['High'].rolling(window=14, min_periods=1).max() # 14일 중 최고가 
    df['ndays_low'] = df['Low'].rolling(window=14, min_periods=1).min() # 14일 중 최저가 
    df['fast_k'] = (df['Close'] - df['ndays_low']) / (df['ndays_high'] - df['ndays_low']) * 100 # Fast %K 구하기 
    df['slow_d'] = df['fast_k'].rolling(window=3).mean() # Slow %D 구하기

    # MFI 구하기 
    df['PB'] = (df['Close'] - df['lower']) / (df['upper'] - df['lower']) 
    df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3 
    df['PMF'] = 0 
    df['NMF'] = 0 
    for i in range(len(df.Close)-1): 
        if df.TP.values[i] < df.TP.values[i+1]:
            df.PMF.values[i+1] = df.TP.values[i+1] * df.Volume.values[i+1]
            df.NMF.values[i+1] = 0 
        else: 
            df.NMF.values[i+1] = df.TP.values[i+1] * df.Volume.values[i+1] 
            df.PMF.values[i+1] = 0 

    df['MFR'] = (df.PMF.rolling(window=10).sum() / df.NMF.rolling(window=10).sum()) 
    df['MFI10'] = 100 - 100 / (1 + df['MFR'])

    #역추세전략을 위한 IIP계산 
    df['II'] = (2*df['Close']-df['High']-df['Low'])/(df['High']-df['Low'])*df['Volume'] 
    df['IIP21'] = df['II'].rolling(window=21).sum()/df['Volume'].rolling(window=21).sum()*100

    #RSI 구하기 
    U = np.where(df['Close'].diff(1) > 0, df['Close'].diff(1), 0) 
    D = np.where(df['Close'].diff(1) < 0, df['Close'].diff(1) *(-1), 0) 
    AU = pd.DataFrame(U, index=df.index).rolling(window=14).mean() 
    AD = pd.DataFrame(D, index=df.index).rolling(window=14).mean() 
    RSI = AU / (AD+AU) *100 
    df['RSI'] = RSI

    #이동평균선 생성
    ma5 = go.Scatter(x=df.index, y=df['ma5'], line=dict(color='red', width=0.8), name='ma5',legendgroup='ma', legendgrouptitle_text='ma%')
    # ma20 = go.Scatter(x=df.index, y=df['MA20'], line=dict(color='red', width=0.9), name='ma20')
    ma20 = go.Scatter(x=df.index, y=df['ma20'], line=dict(color='black', width=2), name='ma20',legendgroup='ma', legendgrouptitle_text='ma%') 
    ma60 = go.Scatter(x=df.index, y=df['ma60'], line=dict(color='green', width=1), name='ma60',legendgroup='ma', legendgrouptitle_text='ma%')
    # df = df[25:]
    # # 위의 보조지표들이 초기 시작일부터 최대 25일까지 표시되지 않기 때문입니다.

    candle = go.Candlestick(x=df.index,open=df['Open'],high=df['High'],low=df['Low'],close=df['Close'], increasing_line_color = 'red',decreasing_line_color = 'blue', showlegend=False) 
    upper = go.Scatter(x=df.index, y=df['upper'], line=dict(color='skyblue', width=2), name='bol_upper',legendgroup='bol', legendgrouptitle_text='Bollinger Band') 
    lower = go.Scatter(x=df.index, y=df['lower'], line=dict(color='skyblue', width=2), name='bol_lower',legendgroup='bol', legendgrouptitle_text='Bollinger Band') 

    
    Volume = go.Bar(x=df.index, y=df['Volume'], marker_color='red', name='Volume', showlegend=False) 
    MACD = go.Scatter(x=df.index, y=df['MACD'], line=dict(color='blue', width=2), name='MACD', legendgroup='group2', legendgrouptitle_text='MACD') 
    MACD_Signal = go.Scatter(x=df.index, y=df['MACD_Signal'], line=dict( color='orange', width=2), name='MACD_Signal') 
    MACD_Oscil = go.Bar(x=df.index, y=df['MACD_Oscil'], marker_color='purple', name='MACD_Oscil')
    fast_k = go.Scatter(x=df.index, y=df['fast_k'], line=dict(color='green', width=2), name='fast_k', legendgroup='group3', legendgrouptitle_text='%K %D') 
    slow_d = go.Scatter(x=df.index, y=df['slow_d'], line=dict(dash='dashdot', color='black', width=2), name='slow_d') 
    PB = go.Scatter(x=df.index, y=df['PB']*100, line=dict(color='blue', width=2), name='PB', legendgroup='group4', legendgrouptitle_text='PB, MFI') 
    MFI10 = go.Scatter(x=df.index, y=df['MFI10'], line=dict(dash='dashdot', color='green', width=2), name='MFI10') 
    RSI = go.Scatter(x=df.index, y=df['RSI'], line=dict(color='red', width=2), name='RSI', legendgroup='group5', legendgrouptitle_text='RSI')

    # 스타일 
    fig = ms.make_subplots(rows=3, cols=2, specs=[[{'rowspan':2},{}],[None,{}],[{},{}]], shared_xaxes=True,shared_yaxes=False, horizontal_spacing=0.05, vertical_spacing=0.02) 

    fig.add_trace(candle,row=1,col=1) 
    fig.add_trace(upper,row=1,col=1) 
    fig.add_trace(lower,row=1,col=1) 
   
    fig.add_trace(ma5,row=1,col=1) 
    fig.add_trace(ma20,row=1,col=1)
    fig.add_trace(ma60,row=1,col=1)  
    

    fig.add_trace(Volume,row=3,col=1) 

    # fig.add_trace(candle,row=1,col=2) 
    # fig.add_trace(upper,row=1,col=2) 
    # fig.add_trace(ma20,row=1,col=2) 
    # fig.add_trace(lower,row=1,col=2) 

    fig.add_trace(MACD,row=1,col=2) 
    fig.add_trace(MACD_Signal,row=1,col=2) 
    fig.add_trace(MACD_Oscil,row=1,col=2) 

    fig.add_trace(fast_k,row=2,col=2) 
    fig.add_trace(slow_d,row=2,col=2) 

    # fig.add_trace(PB,row=4,col=2) 
    # fig.add_trace(MFI10,row=4,col=2)

    fig.add_trace(RSI,row=3,col=2)

    # # 추세추종 
    # for i in range(len(df['Close'])): 
    #     if df['PB'][i] > 0.8 and df['MFI10'][i] > 80: 
    #         trend_fol = go.Scatter(x=[df.index[i]], y=[df['Close'][i]], marker_color='orange', marker_size=20, marker_symbol='triangle-up', opacity=0.7, showlegend=False) 
    #         fig.add_trace(trend_fol,row=1,col=1) 
    #     elif df['PB'][i] < 0.2 and df['MFI10'][i] < 20: 
    #         trend_fol = go.Scatter(x=[df.index[i]], y=[df['Close'][i]], marker_color='darkblue', marker_size=20, marker_symbol='triangle-down', opacity=0.7, showlegend=False)
    #         fig.add_trace(trend_fol,row=1,col=1)
            
    # # 역추세추종 
    # for i in range(len(df['Close'])):
    #     if df['PB'][i] < 0.05 and df['IIP21'][i] > 0:
    #         trend_refol = go.Scatter(x=[df.index[i]], y=[df['Close'][i]], marker_color='purple', marker_size=20, marker_symbol='triangle-up', opacity=0.7, showlegend=False) #보라 
    #         fig.add_trace(trend_refol,row=1,col=1) 
    #     elif df['PB'][i] > 0.95 and df['IIP21'][i] < 0: 
    #         trend_refol = go.Scatter(x=[df.index[i]], y=[df['Close'][i]], marker_color='skyblue', marker_size=20, marker_symbol='triangle-down', opacity=0.7, showlegend=False) #하늘 
    #         fig.add_trace(trend_refol,row=1,col=1)
    # fig.add_trace(trend_refol,row=1,col=1)
    # 추세추총전략을 통해 캔들차트에 표시합니다.
    # fig.add_trace(trend_refol,row=1,col=1)
    # 역추세 전략을 통해 캔들차트에 표시합니다.


    # Plotly 레이아웃
    fig.update_layout(autosize=True, xaxis1_rangeslider_visible=False, xaxis2_rangeslider_visible=False, margin=dict(l=50,r=50,t=50,b=50), template='seaborn', 
                   width=1500, height=800,  xaxis1=dict(rangeselector=dict(
                                                                            buttons=list([
                                                                                dict(count=30, label="30d", step="day", stepmode="backward"),
                                                                                dict(count=6, label="6m", step="month", stepmode="backward"),
                                                                                dict(count=1, label="1y", step="year", stepmode="backward"),
                                                                                dict(step="all") ])     )        ) ,
                                            xaxis2=dict(rangeselector=dict(
                                                                            buttons=list([
                                                                                dict(count=30, label="30d", step="day", stepmode="backward"),
                                                                                dict(count=6, label="6m", step="month", stepmode="backward"),
                                                                                dict(count=1, label="1y", step="year", stepmode="backward"),
                                                                                dict(step="all") ])     )        )) 
    #[추세추종전략:오↑파↓] [역추세전략:보↑하↓]    # title=f'{stock_name}({str(ticker)})의 매수날짜{stock_from} 매도날짜{stock_to}',
    fig.update_xaxes(tickformat='%y년%m월%d일', zeroline=True, zerolinewidth=2, zerolinecolor='black', showgrid=True, gridwidth=2, gridcolor='lightgray', showline=True,linewidth=2, linecolor='black', mirror=True) 
    fig.update_yaxes(tickformat=',d', zeroline=True, zerolinewidth=2, zerolinecolor='black', showgrid=True, gridwidth=2, gridcolor='lightgray',showline=True,linewidth=2, linecolor='black', mirror=True) 
    fig.update_traces(xhoverformat='%y년%m월%d일') 
    
    # fig.update_xaxes(
    # rangeslider_visible=False,
    # rangeselector=dict(
    #     buttons=list([
    #         dict(count=30, label="30d", step="day", stepmode="backward"),
    #         dict(count=6, label="6m", step="month", stepmode="backward"),
    #         dict(count=1, label="1y", step="year", stepmode="backward"),
    #         dict(step="all") ])     )        )

    config = dict({'scrollZoom': True}) 



    fig.show(config=config)
    웹 = fig.write_html(''+stock_name+'.html')
    return 웹


In [ ]:
plotly차트(stock_name='삼성전자',stock_from='20190101',stock_to='20201231',ticker='005930')